In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

#### Load Data

In [ ]:
df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)
print(df_train.shape, df_test.shape)


In [ ]:
layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

In [ ]:
cond = (df_train['layer_2'] == 10) & (df_train['layer_3'] == 10) & (df_train['layer_4'] == 10)
df_model = df_train[cond]
df_model.shape

In [ ]:
# size = 48
W = 226 # input_volume_size
F = 3  # kernel_size
S = 1   # strides
P = 1
# padding_size

size = (W - F + 2*P) / S + 1
size
# ((size - 1) * S) - 2*P + F

#### Model

In [ ]:
class CNNModel(torch.nn.Module):
    def __init__(self, dropout_probability=0.5):
        super().__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.cnn = torch.nn.Sequential(
            torch.nn.Conv1d(1, 12, 3, stride=1, padding=1), torch.nn.BatchNorm1d(12), relu,
            torch.nn.Conv1d(12, 12, 3, stride=1, padding=1), torch.nn.BatchNorm1d(12), relu,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(12, 24, 3, stride=1, padding=1), torch.nn.BatchNorm1d(24), relu,
            torch.nn.Conv1d(24, 24, 3, stride=1, padding=1), torch.nn.BatchNorm1d(24), relu,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(24, 48, 3, stride=1, padding=1), torch.nn.BatchNorm1d(48), relu,
            torch.nn.Conv1d(48, 48, 3, stride=1, padding=1), torch.nn.BatchNorm1d(48), relu,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(48, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), relu,
            torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), relu,
            torch.nn.MaxPool1d(2),
            torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), relu,
            torch.nn.Conv1d(96, 96, 3, stride=1, padding=1), torch.nn.BatchNorm1d(96), relu,
            torch.nn.MaxPool1d(2),
        )
            
        self.clf = torch.nn.Sequential(
            torch.nn.Linear(672, 512), relu, dropout, #torch.nn.BatchNorm1d(128), dropout,
            torch.nn.Linear(512, 512), relu, dropout, 
            torch.nn.Linear(512, 4)
        )
        
    def forward(self, x):
        x = x.unsqueeze(1)
        out = self.cnn(x)
        dim = 1
        for d in out.size()[1:]: #24, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.clf(out)
        return out

#         return self.model(x)
    
class SemiDataset(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
        self.y = df[y_cols].values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
    

#### Trainer

In [ ]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer, scheduler, device):
        self.device = device
        self.model = model#.to(self.device)
        self.criterion = criterion#.to(self.device)
        self.optimizer = optimizer
        self.scheduler = scheduler

        print(self.model.train())
        pass
    
    def train(self, data_loader):
        self.model.train()
        total_loss = 0
#         print('train_loader', len(train_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            y_pred = self.model(X_batch)
#             print(y_pred, y_batch)
            
#             y_pred = torch.round(y_pred)    
            loss = self.criterion(y_pred, y_batch)
            total_loss = total_loss + loss.item()

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
        
#         print(f'total_loss {total_loss / len(data_loader)}')
        return total_loss / len(data_loader)
    
    def eval(self, data_loader):
        self.model.eval()
        total_loss = 0
#         print('valid_loader', len(valid_loader))
        for data in data_loader:
            X_batch, y_batch = data
            X_batch = X_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            with torch.no_grad():
                y_pred = self.model(X_batch)
                loss = self.criterion(y_pred, y_batch)
                
                y_pred = torch.round(y_pred)
                y_pred = y_pred / 10
                y_pred = torch.round(y_pred)
                y_pred = y_pred * 10
                
                print(y_pred, y_batch)
#                 print(loss.item())
                total_loss = total_loss + loss.item()
#         print(f'total_loss {total_loss / len(data_loader)}')
        return total_loss / len(data_loader)

    def save(self, model_path='checkpoint.pt'):
        torch.save(self.model.state_dict(), 'checkpoint.pt')
        return
    
    def load(self, model_path='checkpoint.pt'):
        self.model.load_state_dict(torch.load(model_path))
        return

#### Train

In [ ]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print(model_ts)

batch_size = 10000
num_workers = 0

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'fea_size {len(fea_cols)} layer_cols {layer_cols}')


# model = DNNModel(input_size=len(fea_cols), dropout_probability=0.5).to(device)
model = CNNModel(dropout_probability=0.5).to(device)

    
criterion = nn.L1Loss(reduction='mean').to(device)
# criterion = nn.MSELoss(reduction='mean').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, )
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

scheduler = StepLR(optimizer, step_size=200, gamma=0.9)

trainer = Trainer(model, criterion, optimizer, scheduler, device)
# trainer.load()

# train_dataset = SemiDataset(df_model[fea_cols + layer_cols], fea_cols, layer_cols)
train_dataset = SemiDataset(df_model[fea_cols + layer_cols], fea_cols, layer_cols)

train_loader_params = {
    'dataset' : train_dataset,
    'batch_size' : batch_size,
    'shuffle' : False,
    'num_workers' : num_workers,
    'drop_last' : False,
}
train_loader = DataLoader(**train_loader_params)

print(f'train_loader {len(train_loader)}')

In [ ]:
# model.load_state_dict(torch.load('checkpoint.pt'))

In [ ]:
total_epoch = 5000

for e in tqdm_notebook(range(total_epoch), total=total_epoch, desc='Epoch'):
    loss = trainer.train(train_loader)
    
    if e % 100 == 0:
        ts = datetime.now().strftime('%Y%m%dT%H%M%S')
        lr = optimizer.param_groups[0]['lr']
        print(f'[{ts}] Epock {e} / {total_epoch}\tlr:{lr}\tloss: {loss}')
    
torch.save(model.state_dict(), 'checkpoint.pt')

In [ ]:
# trainer = Trainer(model, criterion, optimizer, None, device)
loss = trainer.eval(train_loader)

In [ ]:
loss